In [13]:
import psi_status_tag

In [19]:
from abc import ABC, abstractmethod
import do_psi
import ds_psi
from psi_abs import PsiProtocol


class DuetRole(ABC): 
    def __init__(self, data_dir=None, label_dir=None, index_dir=None, name=""):
        
        #load data
        #self.index_list = self.__load_data(index_dir) 
        #data_list = self.__load_data(data_dir) 
        #label_list = self.__load_data(label_dir) 
        
        #create dataset
        #self.dataset = self.__create_dataset(data_list, label_list, self.index_list)
        
        self.name = name
        self.duet = None
        self._protocol = None
        
    def __load_data(self, directory):
        return pickle.load(open(directory, 'rb'), encoding='utf-8') if directory else None
    
    def __create_dataset(self, data_list, label_list, index_list):
        if not index_list: 
            label_set = torch.Tensor(label_list) if label_list else None
            bs = torch.Tensor(data_list) if data_list else None
            dataset = SampleSetWithLabelsNoIndex(label_set, bs) if (label_set and bs) else (label_set or bs)

        else: #there is index
            label_set = BaseIndexSet(index_list, self.name, label_list, is_labels=True) if label_list else None
            bs = BaseIndexSet(index_list, self.name, data_list, is_labels=False) if data_list else None
            dataset = SampleIndexSetWithLabels(label_set, bs, self.name) if (label_set and bs) else (label_set or bs)
            
        return dataset
        
    @abstractmethod
    def connect_to_duet(self, loopback=True, server_id=None):
        pass
    
    @property
    def protocol(self):
        return self._protocol
    
    @protocol.setter
    def protocol(self, psi_prot):
        assert type(psi_prot) == PsiProtocol, "The protocol should be a PSI Protocol"
        self._protocol = psi_prot


class DataScientist(DuetRole):        
    def connect_to_duet(self, loopback=True, server_id=None):
        self.duet = sy.join_duet(loopback=loopback) if loopback else sy.join_duet(server_id)
        return self.duet


class DataOwner(DuetRole):    
    def connect_to_duet(self, loopback=True):
        self.duet = sy.launch_duet(loopback=loopback)
        return self.duet 

In [20]:
DataOwner()

In [41]:
from torch.utils.data import Dataset 
from abc import ABC

class IndexSet(ABC): 
    def __init__(self,ids,owner):
        self.ids = ids
        self.owner = owner
        
    @property
    def ids(self):
        return self.__ids
    
    @ids.setter
    def ids(self, ids):
        self.__ids = ids
        
    
    
class BaseIndexSet(IndexSet, Dataset):
    def __init__(self, ids, owner, is_labels=False):
        self.values_dic = {}
        super(BaseIndexSet, self).__init__(ids, owner)

In [42]:
BaseIndexSet([1,2,3], "j")